**This notebook uses the embeddings generated in https://www.kaggle.com/code/mohammedobeidat/product-embeddings-from-images-keras-vgg16/notebook (version 3) to find top 20 similar items.**

Data can be found in this public dataset https://www.kaggle.com/datasets/mohammedobeidat/hm-product-image-embeddings

**Similar items are saved in a pickle file in https://www.kaggle.com/datasets/mohammedobeidat/hm-top-20-similar-items-using-image-embeddings**

**article_id's are saved in another pcikle in the same path sorted in the same order as the embeddings.**

In [ ]:
import pickle
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
f = open('../input/hm-product-image-embeddings/embeds.pickle', 'rb')
paths = open('../input/hm-product-image-embeddings/paths.pickle', 'rb')
labels = open('../input/hm-product-image-embeddings/labels.pickle', 'rb')
paths = pickle.load(paths)

In [ ]:
ids = np.array([path[-14:-4] for path in paths])

In [ ]:
labels = np.array([pickle.load(labels) for i in range(105100)])

In [ ]:
embeds = np.array([pickle.load(f)[0] for i, path in zip(range(105100), paths)])

In [ ]:
df = pd.DataFrame(embeds)
df['article_id'] = ids
df['label'] = labels

In [ ]:
df[['article_id', 'label']].to_csv('cnn_labels.csv', index=False)

In [ ]:
x = df.drop(['article_id', 'label'], axis=1)
y = df.article_id

In [ ]:
knn = KNN(20, metric='cosine')

In [ ]:
knn.fit(x,y)

In [ ]:
# scores = knn.kneighbors()

In [ ]:
# similar = open('top_20.pickle', 'wb')
# pickle.dump(scores, similar)

In [ ]:
def plot_items(items, scores, labels):
    path = "../input/h-and-m-personalized-fashion-recommendations/images"

    k = len(items)
    fig = plt.figure(figsize=(3*k, 10))
    for item, i, score, label in zip(items, range(1, k+1), scores, labels):
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, k, i)
        plt.axis(False)
        plt.title(str(score)+' - '+(label))
        plt.imshow(image)

In [ ]:
%%time
rand = x.sample(1)
sample = knn.kneighbors(rand, 20)
rcmnds = ids[sample[1][0]]
scores = np.round(1- sample[0][0], 2)
sample_labels = df[df.article_id.isin(rcmnds)].label
print(scores.mean())
plot_items(rcmnds, scores, labels)